# 3D LIDAR Object Detection using TAO PointPillars

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. 

Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

<img align="center" src="https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png" width="1080"> 

## Sample prediction of PointPillars
<img align="center" src="https://github.com/vpraveen-nv/model_card_images/blob/main/cv/notebook/common/sample_2.png?raw=true_2.png" width="720"> 

 ## Learning Objectives
In this notebook, you learn how to leverage the simplicity and convenience of TAO to:

* Train a PointPillars model on the KITTI dataset
* Prune the trained model
* Retrain the pruned model to recover lost accuracy
* Run evaluation & inference on the trained model to verify the accuracy
* Export & deploy the model in TensorRT

At the end of this notebook, you will have generated a trained and optimized `PointPillars` model, 
which you may deploy with TensorRT. For more information on how to deploy this model, visit this
[GitHub](https://github.com/NVIDIA-AI-IOT/viz_3Dbbox_ros2_pointpillars) repository sample.

 ### Table of Contents

 This notebook shows an example use case of PointPillars using Train Adapt Optimize (TAO) Toolkit.

 0. [Set up env variables and map drives](#head-0)
 1. [Install the TAO launcher](#head-1)
 2. [Prepare dataset and pretrained model](#head-2)<br>
     2.1 [Download the dataset](#head-2-1)<br>
     2.2 [Verify the downloaded dataset](#head-2-2)<br>
     2.3 [Convert dataset to required format](#head-2-3)<br>
 3. [Provide training specification](#head-3)
 4. [Run TAO training](#head-4)
 5. [Evaluate trained models](#head-5)
 6. [Prune trained models](#head-6)
 7. [Retrain pruned models](#head-7)
 8. [Evaluate retrained model](#head-8)
 9. [Visualize inferences](#head-9)
 10. [Deploy](#head-10)

 ## 0. Set up env variables and map drives <a class="anchor" id="head-0"></a>
 
The following notebook requires you to set an env variable called the `$LOCAL_PROJECT_DIR` as the path to the your workspace. More information on how to set up the dataset and the supported steps in the TAO workflow are provided in the subsequent cells.

In [ ]:
# Setting up env variables for cleaner command line commands.
import os

print("Please replace the variables with your own.")
%env KEY=tlt_encode

# Define this local project directory that must be mapped to the TAO Docker session.
%env LOCAL_PROJECT_DIR=/path/to/tao-experiments
os.environ["LOCAL_DATA_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "data/pointpillars"
)
os.environ["LOCAL_EXPERIMENT_DIR"] = os.path.join(
    os.getenv("LOCAL_PROJECT_DIR", os.getcwd()),
    "pointpillars"
)
!mkdir -p $LOCAL_EXPERIMENT_DIR
!mkdir -p $LOCAL_DATA_DIR
%env USER_EXPERIMENT_DIR=/workspace/tao-experiments/pointpillars
%env DATA_DOWNLOAD_DIR=/workspace/tao-experiments/data/pointpillars
# The sample spec files are present in the same path as the downloaded samples.
# Set this path if you don't run the notebook from the samples directory.
# %env NOTEBOOK_ROOT=~/tao-samples/pointpillars
os.environ["LOCAL_SPECS_DIR"] = os.path.join(
    os.getenv("NOTEBOOK_ROOT", os.getcwd()),
    "specs"
)
%env SPECS_DIR=/workspace/tao-experiments/pointpillars/specs

# Showing list of specification files.
!ls -rlt $LOCAL_SPECS_DIR

The cell below maps the project directory on your local host to a workspace directory in the TAO Docker instance, so that the data and the results are mapped from in and out of the Docker. For more information, see the [launcher instance](https://docs.nvidia.com/tao/tao-toolkit/text/tao_launcher.html) in the user guide.

When running this cell on AWS, update the drive_map entry with the dictionary defined below, so that you don't have permission issues when writing data into folders created by the TAO Docker.

```json
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ],
    "DockerOptions": {
        "user": "{}:{}".format(os.getuid(), os.getgid())
    },
    # set gpu index for tao-converter
    "Envs": [
        {"variable": "CUDA_VISIBLE_DEVICES", "value": os.getenv("GPU_INDEX")},
    ]
}
```

In [ ]:
# Mapping up the local directories to the TAO Docker.
import json
import os
mounts_file = os.path.expanduser("~/.tao_mounts.json")

# Define the dictionary with the mapped drives
drive_map = {
    "Mounts": [
        # Mapping the data directory
        {
            "source": os.environ["LOCAL_PROJECT_DIR"],
            "destination": "/workspace/tao-experiments"
        },
        # Mapping the specs directory.
        {
            "source": os.environ["LOCAL_SPECS_DIR"],
            "destination": os.environ["SPECS_DIR"]
        },
    ]
}

# Writing the mounts file.
with open(mounts_file, "w") as mfile:
    json.dump(drive_map, mfile, indent=4)

In [ ]:
!cat ~/.tao_mounts.json

## 1. Install the TAO launcher <a class="anchor" id="head-1"></a>
The TAO launcher is a Python package distributed as a Python wheel listed in PyPI. You install the launcher by executing the following cell.

The TAO Toolkit recommend that you run the TAO launcher in a virtual env with Python 3.6.9. You can follow the instruction in this [page](https://virtualenvwrapper.readthedocs.io/en/latest/install.html) to set up a Python virtual env using the `virtualenv` and `virtualenvwrapper` packages. After you have setup virtualenvwrapper, set the version of Python to be used in the virtual env by using the `VIRTUALENVWRAPPER_PYTHON` variable. You may do so by running

```sh
export VIRTUALENVWRAPPER_PYTHON=/path/to/bin/python3.x
```
where x >= 6 and <= 8

We recommend performing this step first and then launching the notebook from the virtual environment. In addition to installing TAO Python package, you must meet the following software requirements:
* python >=3.7, <=3.10.x
* docker-ce > 19.03.5
* docker-API 1.40
* nvidia-container-toolkit > 1.3.0-1
* nvidia-container-runtime > 3.4.0-1
* nvidia-docker2 > 2.5.0-1
* nvidia-driver > 460+

Once you have installed the pre-requisites, log in to the Docker registry nvcr.io by following the command below:

```sh
docker login nvcr.io
```

You are prompted to enter a username and password. The username is `$oauthtoken` and the password is the API key generated from `ngc.nvidia.com`. Follow the instructions in the [NGC setup guide](https://docs.nvidia.com/ngc/ngc-overview/index.html#generating-api-key) to generate your own API key.

In [ ]:
# Skip this step if you have already installed the TAO launcher.
!pip3 install --upgrade nvidia-tao

In [ ]:
# View the versions of the TAO launcher
!tao info --verbose

 ## 2. Prepare dataset and pretrained model <a class="anchor" id="head-2"></a>

 We will be using the KITTI detection dataset for the tutorial. To find more details, see
 http://www.cvlibs.net/datasets/kitti/eval_object.php?obj_benchmark=2d. Download the KITTI detection images (http://www.cvlibs.net/download.php?file=data_object_image_2.zip), labels(http://www.cvlibs.net/download.php?file=data_object_label_2.zip), velodyne LIDAR pointcloud(http://www.cvlibs.net/download.php?file=data_object_velodyne.zip), and LIDAR calibration file(http://www.cvlibs.net/download.php?file=data_object_calib.zip) to $DATA_DOWNLOAD_DIR.
 
The data is extracted with the following structure:

```bash
│── ImageSets
│── training
│   ├──calib & velodyne & label_2 & image_2
│── testing
│   ├──calib & velodyne & image_2
```

The `testing` directory is not be used in this notebook because it has no labels. For the `training` dataset, a script is used to do data preprocessing and split it into `train` and `val` splits. Finally, verify that the directory seen by TAO PointPillars looks like:

```bash
│── train
│   ├──lidar & label
│── val
│   ├──lidar & label
```

You can also use this notebook with your own dataset. To use this example with your own dataset, use the same directory structure as mentioned below.

### 2.1 Download the dataset <a class="anchor" id="head-2-1"></a>

Once you have gotten the download links in your email, populate them in place of the `KITTI_IMAGES_DOWNLOAD_URL`,  `KITTI_LABELS_DOWNLOAD_URL`, `KITTI_LIDAR_DOWNLOAD_DIR`, and `KITTI_CALIB_DOWNLOAD_DIR`. This next cell, downloads the data and place in `$LOCAL_DATA_DIR`.

Note that images are only required for KITTI dataset in this notebook, but not required for a general dataset that follows TAO PointPillars standard format. The reason that we need images in the KITTI dataset is that the KITTI dataset does not conform with the standard format and some pre-processing is necessary for it. The preprocessing reads each image's size and retrieves only points that are in the field-of-view(FOV) of the camera from the original LiDAR files. The retrieved FOV-only points are saved to a new LiDAR file for each of the original LiDAR files. This is necessary because the KITTI dataset only has labels in the FOV of camera, but no labels for points outside of camera FOV.

In [ ]:
import os
!mkdir -p $LOCAL_DATA_DIR

os.environ["URL_IMAGES"]="KITTI_IMAGES_DOWNLOAD_URL"
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then wget $URL_IMAGES -O $LOCAL_DATA_DIR/data_object_image_2.zip; else echo "image archive already downloaded"; fi 

os.environ["URL_LABELS"]="KITTI_LABELS_DOWNLOAD_URL"
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then wget $URL_LABELS -O $LOCAL_DATA_DIR/data_object_label_2.zip; else echo "label archive already downloaded"; fi

os.environ["URL_LIDAR"]="KITTI_LIDAR_DOWNLOAD_URL"
!if [ ! -f $LOCAL_DATA_DIR/data_object_velodyne.zip ]; then wget $URL_LIDAR -O $LOCAL_DATA_DIR/data_object_velodyne.zip; else echo "velodyne archive already downloaded"; fi 

os.environ["URL_CALIB"]="CALIB_DOWNLOAD_URL"
!if [ ! -f $LOCAL_DATA_DIR/data_object_calib.zip ]; then wget $URL_CALIB -O $LOCAL_DATA_DIR/data_object_calib.zip; else echo "calib archive already downloaded"; fi 

### 2.2 Verify the downloaded dataset <a class="anchor" id="head-2-2"></a>

In [ ]:
# Check the dataset is present
!mkdir -p $LOCAL_DATA_DIR
!if [ ! -f $LOCAL_DATA_DIR/data_object_image_2.zip ]; then echo 'Image zip file not found, please download.'; else echo 'Found Image zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/data_object_label_2.zip ]; then echo 'Label zip file not found, please download.'; else echo 'Found Labels zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/data_object_velodyne.zip ]; then echo 'Velodyne zip file not found, please download.'; else echo 'Found Velodyne zip file.';fi
!if [ ! -f $LOCAL_DATA_DIR/data_object_calib.zip ]; then echo 'Calib zip file not found, please download.'; else echo 'Found Calib zip file.';fi

In [ ]:
# unpack 
!unzip -u $LOCAL_DATA_DIR/data_object_image_2.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/data_object_label_2.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/data_object_velodyne.zip -d $LOCAL_DATA_DIR
!unzip -u $LOCAL_DATA_DIR/data_object_calib.zip -d $LOCAL_DATA_DIR

### 2.3 Convert dataset to required format<a class="anchor" id="head-2-3"></a>

In [ ]:
# Create output directories
!mkdir -p $LOCAL_DATA_DIR/train/lidar
!mkdir -p $LOCAL_DATA_DIR/train/label
!mkdir -p $LOCAL_DATA_DIR/val/lidar
!mkdir -p $LOCAL_DATA_DIR/val/label

In [ ]:
# Retrieve FOV-only LIDAR points from 360-degree LIDAR points
# Since only FOV data is labelled in KITTI dataset
!tao model pointpillars run python $SPECS_DIR/gen_lidar_points.py -p $DATA_DOWNLOAD_DIR/training/velodyne \
                                                           -c $DATA_DOWNLOAD_DIR/training/calib    \
                                                           -i $DATA_DOWNLOAD_DIR/training/image_2  \
                                                           -o $DATA_DOWNLOAD_DIR/train/lidar

In [ ]:
# Convert labels from Camera coordinate system to LIDAR coordinate system, etc
!tao model pointpillars run python $SPECS_DIR/gen_lidar_labels.py -l $DATA_DOWNLOAD_DIR/training/label_2 \
                                                           -c $DATA_DOWNLOAD_DIR/training/calib \
                                                           -o $DATA_DOWNLOAD_DIR/train/label

In [ ]:
# Drop DontCare class
!tao model pointpillars run python $SPECS_DIR/drop_class.py $DATA_DOWNLOAD_DIR/train/label DontCare

In [ ]:
# train/val split
!tao model pointpillars run python $SPECS_DIR/kitti_split.py $SPECS_DIR/val.txt \
                                                      $DATA_DOWNLOAD_DIR/train/lidar \
                                                      $DATA_DOWNLOAD_DIR/train/label \
                                                      $DATA_DOWNLOAD_DIR/val/lidar \
                                                      $DATA_DOWNLOAD_DIR/val/label

In [ ]:
# Generate dataset statistics for data augmentation
!tao model pointpillars dataset_convert -e $SPECS_DIR/pointpillars.yaml results_dir=$USER_EXPERIMENT_DIR

 ## 3. Provide training specification <a class="anchor" id="head-3"></a>

In [ ]:
!cat $LOCAL_SPECS_DIR/pointpillars.yaml

 ## 4. Run TAO training <a class="anchor" id="head-4"></a>
 * Provide the sample spec file for training.

In [ ]:
!tao model pointpillars train -e $SPECS_DIR/pointpillars.yaml \
                              results_dir=$USER_EXPERIMENT_DIR \
                              dataset.data_info_path=$USER_EXPERIMENT_DIR/data_info \
                              key=$KEY

In [ ]:
# Uncomment to run multi-gpu training
# !tao model pointpillars train -e $SPECS_DIR/pointpillars.yaml \
#                               results_dir=$USER_EXPERIMENT_DIR \
#                               dataset.data_info_path=$USER_EXPERIMENT_DIR/data_info \
#                               key=$KEY \
#                               train.num_gpus=2 \

 ## 5. Evaluate trained models <a class="anchor" id="head-5"></a>

The evaluation metric in TAO PointPillars is different from that in official metric of KITTI point cloud detection. While KITTI metric considers easy/moderate/hard categories of objects and filters small objects whose sizes are smaller than a threshold, it is only meaningful for KITTI dataset. Instead, TAO PointPillars metric is a general metric that does not classify objects into easy/moderate/hard categories and does not exclude objects in calculation of metric. This makes TAO PointPillars metric a general metric that is applicable to a general dataset. The final result is average precision (AP) and mean average precision (mAP) regardless of its details in computation. Due to this, the TAO PointPillars metric is not comparable with KITTI official metric on KITTI dataset, although they are typically similar.

In [ ]:
!tao model pointpillars evaluate -e $SPECS_DIR/pointpillars.yaml \
                                 results_dir=$USER_EXPERIMENT_DIR \
                                 dataset.data_info_path=$USER_EXPERIMENT_DIR/data_info \
                                 key=$KEY

 ## 6. Prune trained models <a class="anchor" id="head-6"></a>
 * Specify pre-trained model
 * Threshold for pruning
 * A key to save and load the model
 * Output directory to store the model
 
Usually, you must to adjust `prune.pruning_thresh` (threshold) for accuracy and model size trade off. Using a higher threshold gives you smaller model (and thus higher inference speed) but accuracy suffers. The threshold to use depends on the dataset. The threshold value below is just a start point. If the retrain accuracy is good, you can increase this value to get smaller models. Otherwise, lower this value to get better accuracy.

In [ ]:
!tao model pointpillars prune -e $SPECS_DIR/pointpillars.yaml \
                              results_dir=$USER_EXPERIMENT_DIR \
                              dataset.data_info_path=$USER_EXPERIMENT_DIR/data_info \
                              key=$KEY \
                              prune.pruning_thresh=0.1

In [ ]:
!ls -lht $LOCAL_EXPERIMENT_DIR

 ## 7. Retrain pruned models <a class="anchor" id="head-7"></a>
 * Model needs to be re-trained to bring back accuracy after pruning
 * Specify re-training specification

In [ ]:
# Retraining using the pruned model as pretrained weights 
!tao model pointpillars train -e $SPECS_DIR/pointpillars_retrain.yaml \
                              results_dir=$USER_EXPERIMENT_DIR/retrain \
                              dataset.data_info_path=$USER_EXPERIMENT_DIR/data_info \
                              key=$KEY

In [ ]:
# Listing the newly retrained model.
!ls -lht $LOCAL_EXPERIMENT_DIR/retrain

 ## 8. Evaluate retrained model <a class="anchor" id="head-8"></a>

In [ ]:
!tao model pointpillars evaluate -e $SPECS_DIR/pointpillars_retrain.yaml \
                                 results_dir=$USER_EXPERIMENT_DIR/retrain \
                                 dataset.data_info_path=$USER_EXPERIMENT_DIR/data_info \
                                 key=$KEY

 ## 9. Visualize inferences <a class="anchor" id="head-9"></a>
 In this section, we run the inference command on the trained models.

In [ ]:
!tao model pointpillars inference -e $SPECS_DIR/pointpillars_retrain.yaml \
                                  results_dir=$USER_EXPERIMENT_DIR/retrain \
                                  dataset.data_info_path=$USER_EXPERIMENT_DIR/data_info \
                                  key=$KEY \
                                  inference.save_to_file=True

The `inference` command produces visualization of bounding boxes of objects in and rendering of point cloud. This command can be slow due to plots. If you are not going to finish it, you can abort it and check the partially detected results(images) in `$USER_EXPERIMENT_DIR/retrain/detected_boxes`.

 ## 10. Deploy! <a class="anchor" id="head-10"></a>

In [ ]:
# Export in FP16 mode.
# Note that the .onnx model in FP16 mode is the same as in FP32 mode.
!tao model pointpillars run rm -f $USER_EXPERIMENT_DIR/retrain/checkpoint_epoch_80.onnx
!tao model pointpillars export -e $SPECS_DIR/pointpillars_retrain.yaml \
                        dataset.data_info_path=$USER_EXPERIMENT_DIR/data_info \
                        key=$KEY \
                        results_dir=$USER_EXPERIMENT_DIR/retrain \
                        export.save_engine=$USER_EXPERIMENT_DIR/retrain/checkpoint_epoch_80.trt